<a href="https://colab.research.google.com/github/acollin19/comp551/blob/Angele/assignment2_group_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2

## Process:

### Task 1: Data Processing  


1. IMDB Reviews
2. 20 news groups: a multi-class labelled textual dataset


### Task 2: Implement Logistic and Multiclass classifiers

1. Logistic Regression
2. Multiclass Regression


### Task 3: Running Experiments



1.    Most positive/negative Z-scores
2.   Implement:  
    * Binary classification on the IMDB Reviews

  *   Multi-class classification on the 20 news group dataset

3. Binary classification AUROC on IMDB data
  *   Logistic Regression 
  *   KNN

4. Multiclass classification accuracy

5. Compare Accuracy of Models (Plot)







In [10]:
import tensorflow as tf
import pandas as pd
import numpy as np
from joblib import Memory
from sklearn.datasets import load_svmlight_file

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


In [11]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!ls aclImdb
!ls aclImdb/test
!ls aclImdb/train

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  45.1M      0  0:00:01  0:00:01 --:--:-- 45.1M
imdbEr.txt  imdb.vocab	README	test  train
labeledBow.feat  neg  pos  urls_neg.txt  urls_pos.txt
labeledBow.feat  pos	unsupBow.feat  urls_pos.txt
neg		 unsup	urls_neg.txt   urls_unsup.txt


In [9]:
# IMDB train df
X,Y = load_svmlight_file("labeledBow.feat", multilabel = True, zero_based = True)



ValueError: ignored

In [ ]:
# Loading 20 news group dataset
twenty_train = fetch_20newsgroups(subset='train', categories=(['rec.sport.hockey','sci.electronics','talk.politics.guns','sci.space']), remove=(['headers', 'footers', 'quotes']))

# Turn the text content into numerical feature vectors
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(twenty_train.data)
vectors.shape
vectors.nnz / float(vectors.shape[0])
